In [ ]:
!pip install torch
!pip install transformers
!pip install pandas
!pip install datasets
!pip install peft
!pip install scikit-learn
!pip install evaluate
!pip install textstat
!pip install numpy
!pip install sacrebleu sacremoses
!pip install bert_score

In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, set_seed
import pandas as pd
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
import evaluate
import textstat
import numpy as np
from torch.utils.data import DataLoader


In [4]:
set_seed(42)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
MAX_LENGTH = 1024
NAME_MODEL = "LORA_TRY_60_4"
N_BEAM = 4
NAME_RUN = ""
NUM_EPOCHS = 10
# Path directly to dataset in 'data/Final'
PATH_DATASET = "/kaggle/input/test-paper-aied/candidate_full_80.csv"
# Path in folder 'model'
PATH_OUTPUT = ""

In [ ]:
dataset = pd.read_csv(PATH_DATASET, sep="¶", engine='python')

In [7]:
dataset = dataset[["normal", "simplified"]]

In [8]:
dataset.dropna(inplace=True)

In [9]:
dataset_shuffled = dataset.sample(frac=1, random_state=42)

In [10]:
dataset_shuffled.reset_index(drop=True, inplace = True)

In [11]:
dataset_shuffled

,normal,simplified
0,Ares nella religione greca è il figlio di Zeus...,Ares era identificato come il dio della guerr...
1,La bella e la bestia (titolo francese: La bell...,La bella e la bestia è una fiaba scritta da Je...
2,"Ginger Baker, pseudonimo di Peter Edward Baker...",Ginger Baker è stato uno dei batteristi fondam...
3,Teissières-de-Cornet è un comune francese di 2...,Teissières-de-Cornet è un comune francese situ...
4,"William Howard Taft (Cincinnati, 15 settembre ...",William Howard Taft è stato un politico statun...
...,...,...
8006,Ouricuri è un comune del Brasile nello Stato d...,Ouricuri è un comune appartente allo stato del...
8007,La pasta (in ambito tecnico definita come past...,"La pasta è un cibo. È fatta di farina, uovo, a..."
8008,"Benjamin Franklin (Boston, 17 gennaio 1706 - F...",Benjamin Franklin fu uno scienziato e politico...
8009,Vetto (Vèt in dialetto reggiano) è un comune i...,Vetto è un comune dell'Italia. Appartiene alla...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("morenolq/bart-it")


In [ ]:
def filter_by_token_length(row):
    # Column tokenization 
    normal_tokens = tokenizer(row['normal'], truncation=False, return_tensors="pt")
    simplified_tokens = tokenizer(row['simplified'], truncation=False, return_tensors="pt")
    # Check sequence do not exceed limit
    
    return len(normal_tokens.input_ids[0]) <= MAX_LENGTH and len(simplified_tokens.input_ids[0]) <= MAX_LENGTH

# Apply function to dataframe and filter rows
df_filtered_by_token_length = dataset_shuffled[dataset_shuffled.apply(filter_by_token_length, axis=1)]

In [15]:
df_filtered_by_token_length.reset_index(drop=True, inplace = True)

In [16]:
df_filtered_by_token_length

,normal,simplified
0,Ares nella religione greca è il figlio di Zeus...,Ares era identificato come il dio della guerr...
1,La bella e la bestia (titolo francese: La bell...,La bella e la bestia è una fiaba scritta da Je...
2,"Ginger Baker, pseudonimo di Peter Edward Baker...",Ginger Baker è stato uno dei batteristi fondam...
3,Teissières-de-Cornet è un comune francese di 2...,Teissières-de-Cornet è un comune francese situ...
4,"William Howard Taft (Cincinnati, 15 settembre ...",William Howard Taft è stato un politico statun...
...,...,...
7961,Ouricuri è un comune del Brasile nello Stato d...,Ouricuri è un comune appartente allo stato del...
7962,La pasta (in ambito tecnico definita come past...,"La pasta è un cibo. È fatta di farina, uovo, a..."
7963,"Benjamin Franklin (Boston, 17 gennaio 1706 - F...",Benjamin Franklin fu uno scienziato e politico...
7964,Vetto (Vèt in dialetto reggiano) è un comune i...,Vetto è un comune dell'Italia. Appartiene alla...


In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_filtered_by_token_length["normal"],
                 df_filtered_by_token_length["simplified"],
                 test_size=0.2,
                 random_state = 42)

In [18]:
X_train, X_eval, y_train, y_eval = train_test_split(X_train,
                 y_train,
                 test_size=0.2,
                 random_state = 42)

In [19]:
train_dataset = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
eval_dataset = pd.concat([X_eval, y_eval], axis=1).reset_index(drop=True)
test_dataset = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

In [20]:
hf_dataset_train = Dataset.from_pandas(train_dataset)
hf_dataset_eval = Dataset.from_pandas(eval_dataset)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("morenolq/bart-it").to(device)

In [22]:
def preprocess_function(examples):
    inputs = examples['normal']
    targets = examples['simplified']
    model_inputs = tokenizer(inputs, max_length=MAX_LENGTH, padding="max_length", truncation=True, return_tensors="pt").to(device)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=MAX_LENGTH, padding="max_length", truncation=True, return_tensors="pt").to(device)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [23]:
tokenized_datasets_train = hf_dataset_train.map(preprocess_function, batched=True)
tokenized_datasets_eval = hf_dataset_eval.map(preprocess_function, batched=True)

Map:   0%|          | 0/5097 [00:00<?, ? examples/s]

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1275 [00:00<?, ? examples/s]

In [ ]:
config = LoraConfig(
    r=8,         # Rank
    lora_alpha=16,    # Alpha
    target_modules=["q_proj", "v_proj", "k_proj"],  
    bias="none",  
    lora_dropout=0.05,
    task_type="SEQ_2_SEQ_LM"
)

In [25]:
bleu = evaluate.load("bleu")
sari = evaluate.load("sari")

In [ ]:
def compute_metrics(pred):
    
    textstat.set_lang("it")
    
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    input_ids = pred.inputs
    
    pred_ids = np.where(pred_ids != -100, pred_ids, tokenizer.pad_token_id)
    labels_ids = np.where(labels_ids != -100, labels_ids, tokenizer.pad_token_id)
    input_ids = np.where(input_ids != -100, input_ids, tokenizer.pad_token_id)
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    input_str = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
    
    results = bleu.compute(predictions=pred_str, references=label_str)
        
    flesch_reading_ease = sum(textstat.flesch_reading_ease(sent) for sent in pred_str) / len(pred_str)
    
    ref_arr = [[sent] for sent in label_str]
    
    sari_score = sari.compute(sources=input_str, predictions=pred_str, references=ref_arr)

    
    return {
        'bleu': results['bleu'],
        'flesch_reading': flesch_reading_ease,
        'sari': sari_score["sari"]
    }

In [27]:
model_lora = get_peft_model(model, config)

In [28]:
model_lora.num_parameters(only_trainable=True)

663552

In [ ]:
training_args = Seq2SeqTrainingArguments(
    run_name=NAME_RUN,
    overwrite_output_dir=True,
    output_dir=PATH_OUTPUT,#
    eval_strategy="epoch", #
    save_strategy="epoch",#
    learning_rate=2e-5, #
    num_train_epochs=NUM_EPOCHS, #
    per_device_train_batch_size=32,#
    weight_decay=0.01, #
    predict_with_generate=True, 
    generation_num_beams = N_BEAM,
    generation_max_length = MAX_LENGTH,
    save_total_limit = 1, #
    load_best_model_at_end=True, #
    include_inputs_for_metrics = True,
    metric_for_best_model="sari",
    logging_strategy="epoch"
)
                         
trainer = Seq2SeqTrainer(
    model=model_lora,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(NAME_MODEL)